In [18]:
# Change to whatever you prefer if this doesn't work. But you will want interactivty so you can zoom
# in and out of the map.
%matplotlib widget

In [19]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from matplotlib.colors import rgb2hex

# I had issues importing basemap on mpl v3.3 due to the 'dedent' module. I downgraded to 
# v3.2 in order to import. Will maybe try rewriting code using Cartopy, since Basemap is no 
# longer supported (I think).
from mpl_toolkits.basemap import Basemap

# Need these modules for changing directory to reference the shapefile folder
import os
import os.path

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# These are just to try some data visualizations in a 3D scatterplot
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

In [20]:
# Read in industry details dataset, converting zip column to string
df1 = pd.read_csv('data/zbp18detail.txt', 
                  converters={'zip': lambda x: str(x)}, 
                  header=0)

# Keep only the industry sector level data, given by naics codes in the form '11----' or similar. 
# Including lower level sectors would result in a sparse matrix.
# For detail, see https://www.census.gov/eos/www/naics/2017NAICS/2017_NAICS_Manual.pdf (page 26)
df1 = df1[df1['naics'].str.contains('^\d{2}\D{4}$')]

# Drop rows with zip 99999, this is a dummy code
df1 = df1[df1['zip'] != '99999']

# The numeric columns were read as objects because they use 'N' for zeros. Coerce them to numeric data and fill NAs with zeros
df1.iloc[:, 3:] = df1.iloc[:, 3:].apply(pd.to_numeric, errors='coerce')
df1.fillna(0, inplace=True)

# Consolidate all n>5 into a single column and drop all others. We'll just use two features, n<5 and n>5
df1['n>5'] = df1.iloc[:, 4:].sum(axis=1)
df1.drop(df1.columns[4:-1], axis=1, inplace=True)
df1.drop('est', axis=1, inplace=True)

# Pivot the data so we have single columns for each feature
zip_details = df1.pivot(index='zip', columns='naics')

FileNotFoundError: [Errno 2] No such file or directory: 'data/zbp18detail.txt'

In [8]:
# Read in the industry totals dataset, converting zip column to string
df2 = pd.read_csv('data/zbp18totals.txt', 
                  converters={'zip': lambda x: str(x)}, 
                  encoding='latin_1',
                  header=0)

# Set zip column as the index
df2.set_index('zip', inplace=True)

# Drop the row 99999, it is a dummy zip. Aslo drop columns we don't want.
df2.drop('99999', axis=0, inplace=True)
df2.drop(['name', 'emp_nf', 'qp1_nf', 'qp1', 'ap_nf', 'cty_name'], axis=1, inplace=True)

# Merge with the zip_details dataframe from above. This throws a warning because zip_details has a column index, but it works out here.
df2_details = df2.merge(zip_details, how='inner', left_index=True, right_index=True)

C:\Users\Casper.Messmann\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [9]:
# Group by state/city in a multiindex. We will do analysis at the city level, but visualize by zip code.
# Cities have multiple associated zip codes, but each zip code has only one city
city = df2_details.groupby(['stabbr', 'city']).agg('sum')

# Truncate 0 employee rows to 1 employee
city['emp'].replace(0, 1, inplace=True)

# Create relationship columns for each pair of metrics
city['ap per emp'] = city['ap'] / city['emp']
city['ap per est'] = city['ap'] / city['est']
city['emp per est'] = city['emp'] / city['est']

In [10]:
# Normalize the city features
city_n = pd.DataFrame(StandardScaler().fit_transform(city))

# StandardScaler returned a numpy array, so we need to set the index and column names again
city_n.set_index(city.index, drop=False, inplace=True)
city_n.columns = ["n_{}".format(x) for x in city.columns]

In [11]:
# Getting the feature vector for Farminton, MI. Reshaping in order to pass to cosine similarity function
sim_vec = np.array(city_n.loc['MI', 'FARMINGTON']).reshape(1, -1)

# Get cosine similarity between FARMINGTON and every other city and add that as a new column
sims = cosine_similarity(sim_vec, city_n)
city_n['sim'] = sims.reshape(-1, 1)

In [12]:
# Make a new df with just zip, city, state info from our original df2
zips = df2[['city', 'stabbr']]

# Switch the index to match city_n and move zip codes to a column. I get a warning here, but I think it's a false positive. Ignoring.
zips['zips'] = zips.index
zips.set_index(['stabbr', 'city'], inplace=True)

# Merge the dataframes so we get the zip codes for each state/city again
zips = zips.merge(city_n, how='inner', left_index=True, right_index=True)

<ipython-input-12-b2b9509230b6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zips['zips'] = zips.index


In [13]:
# Get the colormap
cmap = cm.get_cmap('coolwarm')

# The similarity values are from [-1, 1], so we first normalize to [0,1]
zips['n_sim'] = zips['sim'].apply(lambda x: (x+1)/2)

# Get the zip code colors and turn into a dictionary of form 'zip: color'
zip_colors = cmap(zips['n_sim'])
zip_colors = dict(zip(zips['zips'], zip_colors))

# We'll just show MI on the map, but some surrounding states will also show up (MI, WI, IL)
# Getting just the zip codes and colors for those states
mapzips = list(zips.loc[['MI', 'WI', 'IL']]['zips'])
mapzips_colors = {z: zip_colors[z] for z in mapzips}

In [14]:
# Changing directory to folder with shapefile data for drawing the map
us_shape_file_dir = 'data/cb_2018_us_zcta510_500k/'
os.chdir(us_shape_file_dir)

In [21]:
# Creating the plot for Michigan
plt.figure(figsize=(9, 5.5))

# Setting the map's bounding coordinates
lowerlon = -90.5
upperlon = -81
lowerlat = 41.689
upperlat = 47.5

# Creating the basemap object
m = Basemap(llcrnrlon=lowerlon,
            llcrnrlat=lowerlat,
            urcrnrlon=upperlon,
            urcrnrlat=upperlat,
            resolution='c',
            projection='lcc',
            lat_0=lowerlat,
            lat_1=upperlat,
            lon_0=lowerlon,
            lon_1=upperlon)

# Read in the shapefile and pull out zip code info
shp_info = m.readshapefile(os.path.basename('/cb_2018_us_zcta510_500k'), 'states', drawbounds=True)
zip_info = m.states_info

# These loops are for getting our zip code list and colors, but we need to get them in the right order
# with the shapefile. I imagine there is a much better way of doing this, because I don't really want 
# to loop through every US zip code.
ziplist = []
colors = {}

for d in zip_info:
    iterzip = d["ZCTA5CE10"]
    if iterzip in mapzips:
        colors[iterzip] = mapzips_colors[iterzip][:3]
    ziplist.append(iterzip)
    
for nshape, seg in enumerate(m.states):
    i, j = zip(*seg)
    if ziplist[nshape] in mapzips:
        color = rgb2hex(colors[ziplist[nshape]])
        edgecolor = '#FFFFFF'
        plt.fill(i, j, color, edgecolor=edgecolor)

# Just adding a colorbar to the plot with descriptive labels
sm = plt.cm.ScalarMappable(cmap=cmap, norm=mpl.colors.Normalize(vmin=0, vmax=1))
mm = plt.cm.ScalarMappable(cmap=cmap)
mm.set_array([0, 1])
cbar = plt.colorbar(mm, ticks=np.arange(0, 2, 1), orientation='vertical')
cbar.set_ticklabels(['Lest Similar', 'Most Similar'])

plt.gca().axis('off')
plt.title('Economic Similarity to Farmington, MI by City Zip Codes')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ProjError: x, y, z, and time must be same size

In [22]:
# Out of curiosity, thes are the 10 most similar cities to Farmington, MI accoring to our methodology
zips['sim'].drop_duplicates().sort_values(ascending=False)[:10]

stabbr  city       
MI      FARMINGTON     1.000000
OH      WESTERVILLE    0.927645
MI      TROY           0.926306
MN      MINNEAPOLIS    0.909998
MI      NOVI           0.909105
OH      DUBLIN         0.907356
FL      TAMPA          0.907048
MD      ROCKVILLE      0.905961
CA      CARLSBAD       0.897065
AL      HUNTSVILLE     0.896974
Name: sim, dtype: float64

In [23]:
# This cell is ONLY for visualizing the data using PCA. We'll just do cities in MI.
# First dedupe the the zips df, because each city has multiple zips, now we only have city data 
# Then we can apply PCA and make a scatterplot. The dot colors represent similarity to Farmington, MI

# First we select just MI and dedupe rows, because each city has multiple zip codes attached to it.
deduped = zips.loc['MI'].drop('zips', axis=1).drop_duplicates()

# Select just the features we want to pass to PCA. Also get the colors for each city.
x = deduped.iloc[:,:6]
x_colors = cmap(deduped['n_sim'])

# Run PCA with 3 components
pca = PCA(n_components=3).fit_transform(x)

# Make the plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pca[:,0], pca[:,1], pca[:,2], c=x_colors);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …